In [162]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation,metrics
from sklearn.grid_search import GridSearchCV
import matplotlib.pylab as plt
%matplotlib inline

In [163]:
train = pd.read_csv('cleaned_train.csv')
target = pd.read_csv('outcome_train.csv')
train.shape

(26729, 11)

In [164]:
from sklearn.preprocessing import LabelEncoder,label_binarize
outcome_le = LabelEncoder()
outcome = outcome_le.fit_transform(target.values)
#outcome = label_binarize(outcome,classes = [0,1,2,3,4,])

//anaconda/lib/python3.5/site-packages/sklearn/preprocessing/label.py:125: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [165]:
outcome[:,0]

IndexError: too many indices for array

Step 1: Fix learning rate and number of estimators for tuning tree-based parameters

In order to decide on boosting parameters, we need to set some initial values of other parameters. Lets take the following values:

max_depth = 5 : This should be between 3-10. I’ve started with 5 but you can choose a different number as well. 4-6 can be good starting points.

min_child_weight = 1 : A smaller value is chosen because it is a highly imbalanced class problem and leaf nodes can have smaller size groups.

gamma = 0 : A smaller value like 0.1-0.2 can also be chosen for starting. This will anyways be tuned later.

subsample, colsample_bytree = 0.8 : This is a commonly used used start value. Typical values range between 0.5-0.9.

scale_pos_weight = 1: Because of high class imbalance.

Lets take the default learning rate of 0.1 here and check the optimum number of trees using cv function of xgboost. 

In [166]:
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softprob',
 scale_pos_weight=1,
 seed=27)
xgb1 = xgb1.fit(train.values,outcome,eval_metric = 'mlogloss')

In [146]:
train_y_prediction = xgb1.predict(train.values)
train_y_perdprob = xgb1.predict_proba(train.values)

In [147]:
metrics.accuracy_score(outcome,train_y_prediction)

0.81091698155561376

In [129]:
param_test1 = {
 'max_depth':list(range(3,10,2)),
 'min_child_weight':list(range(1,6,2))
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'multi:softprob', scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='log_loss',iid=False, cv=5)

In [130]:
gsearch1.fit(train.values,outcome)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: -0.80223, std: 0.00759, params: {'min_child_weight': 1, 'max_depth': 3},
  mean: -0.80201, std: 0.00774, params: {'min_child_weight': 3, 'max_depth': 3},
  mean: -0.80227, std: 0.00757, params: {'min_child_weight': 5, 'max_depth': 3},
  mean: -0.77701, std: 0.00652, params: {'min_child_weight': 1, 'max_depth': 5},
  mean: -0.77750, std: 0.00679, params: {'min_child_weight': 3, 'max_depth': 5},
  mean: -0.77800, std: 0.00672, params: {'min_child_weight': 5, 'max_depth': 5},
  mean: -0.76267, std: 0.00700, params: {'min_child_weight': 1, 'max_depth': 7},
  mean: -0.76390, std: 0.00751, params: {'min_child_weight': 3, 'max_depth': 7},
  mean: -0.76476, std: 0.00768, params: {'min_child_weight': 5, 'max_depth': 7},
  mean: -0.76431, std: 0.00786, params: {'min_child_weight': 1, 'max_depth': 9},
  mean: -0.76400, std: 0.00730, params: {'min_child_weight': 3, 'max_depth': 9},
  mean: -0.76405, std: 0.00851, params: {'min_child_weight': 5, 'max_depth': 9}],
 {'max_depth': 7, 'min_chil

In [132]:
param_test2 = {
 'max_depth':[7,8,9],
 'min_child_weight':[1,2]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=7,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'multi:softprob', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2, scoring='log_loss',iid=False, cv=5)
gsearch2.fit(train.values,outcome)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

([mean: -0.76267, std: 0.00700, params: {'max_depth': 7},
  mean: -0.76181, std: 0.00704, params: {'max_depth': 8},
  mean: -0.76431, std: 0.00786, params: {'max_depth': 9}],
 {'max_depth': 8},
 -0.76180683358796597)

best_params so far:
max_depth: 8
min_child_weight: 1
log_loss = -0.7618

In [141]:
param_test3 = {
 'gamma':[i/10.0 for i in range(9,15)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=8,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'multi:softprob', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring='log_loss',n_jobs=4,iid=False, cv=5)
gsearch3.fit(train.values,outcome)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

//anaconda/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda

([mean: -0.75940, std: 0.00696, params: {'gamma': 0.9},
  mean: -0.76075, std: 0.00653, params: {'gamma': 1.0},
  mean: -0.76040, std: 0.00682, params: {'gamma': 1.1},
  mean: -0.76093, std: 0.00684, params: {'gamma': 1.2},
  mean: -0.76011, std: 0.00687, params: {'gamma': 1.3},
  mean: -0.76131, std: 0.00682, params: {'gamma': 1.4}],
 {'gamma': 0.9},
 -0.75940338547688613)

gamma tested:0~1.4, best:0.9
best score :0.7594

In [142]:
param_test4 = {
 'subsample':[i/100.0 for i in range(75,90,5)],
 'colsample_bytree':[i/100.0 for i in range(75,90,5)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=8,
 min_child_weight=1, gamma=0.9, subsample=0.8, colsample_bytree=0.8,
 objective= 'multi:softprob',nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test4, scoring='log_loss',n_jobs=4,iid=False, cv=5)
gsearch4.fit(train.values,outcome)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

//anaconda/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda

([mean: -0.76156, std: 0.00738, params: {'subsample': 0.75, 'colsample_bytree': 0.75},
  mean: -0.75940, std: 0.00696, params: {'subsample': 0.8, 'colsample_bytree': 0.75},
  mean: -0.76027, std: 0.00688, params: {'subsample': 0.85, 'colsample_bytree': 0.75},
  mean: -0.76156, std: 0.00738, params: {'subsample': 0.75, 'colsample_bytree': 0.8},
  mean: -0.75940, std: 0.00696, params: {'subsample': 0.8, 'colsample_bytree': 0.8},
  mean: -0.76027, std: 0.00688, params: {'subsample': 0.85, 'colsample_bytree': 0.8},
  mean: -0.76204, std: 0.00733, params: {'subsample': 0.75, 'colsample_bytree': 0.85},
  mean: -0.76068, std: 0.00654, params: {'subsample': 0.8, 'colsample_bytree': 0.85},
  mean: -0.75988, std: 0.00640, params: {'subsample': 0.85, 'colsample_bytree': 0.85}],
 {'colsample_bytree': 0.75, 'subsample': 0.8},
 -0.75940338547688613)

best colsample_bytree:0.75
best subsample: 0.8
-0.75940338547688613

In [143]:
param_test5 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100],
 'reg_lambda':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=8,
 min_child_weight=1, gamma=0.9, subsample=0.8, colsample_bytree=0.75,
 objective= 'multi:softprob',nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test5, scoring='log_loss',n_jobs=4,iid=False, cv=5)
gsearch5.fit(train.values,outcome)
gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_

//anaconda/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda

([mean: -0.76135, std: 0.00600, params: {'reg_alpha': 1e-05, 'reg_lambda': 1e-05},
  mean: -0.76088, std: 0.00636, params: {'reg_alpha': 1e-05, 'reg_lambda': 0.01},
  mean: -0.76117, std: 0.00652, params: {'reg_alpha': 1e-05, 'reg_lambda': 0.1},
  mean: -0.75940, std: 0.00696, params: {'reg_alpha': 1e-05, 'reg_lambda': 1},
  mean: -0.78853, std: 0.00769, params: {'reg_alpha': 1e-05, 'reg_lambda': 100},
  mean: -0.76042, std: 0.00667, params: {'reg_alpha': 0.01, 'reg_lambda': 1e-05},
  mean: -0.76043, std: 0.00685, params: {'reg_alpha': 0.01, 'reg_lambda': 0.01},
  mean: -0.76035, std: 0.00743, params: {'reg_alpha': 0.01, 'reg_lambda': 0.1},
  mean: -0.76034, std: 0.00680, params: {'reg_alpha': 0.01, 'reg_lambda': 1},
  mean: -0.78873, std: 0.00755, params: {'reg_alpha': 0.01, 'reg_lambda': 100},
  mean: -0.76015, std: 0.00667, params: {'reg_alpha': 0.1, 'reg_lambda': 1e-05},
  mean: -0.75972, std: 0.00643, params: {'reg_alpha': 0.1, 'reg_lambda': 0.01},
  mean: -0.76037, std: 0.00731, 

In [144]:
param_test5 = {
 'reg_alpha':[1e-10, 1e-8, 1e-5,1e-3],
 'reg_lambda':[0.5, 1,25,50]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=8,
 min_child_weight=1, gamma=0.9, subsample=0.8, colsample_bytree=0.75,
 objective= 'multi:softprob',nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test5, scoring='log_loss',n_jobs=4,iid=False, cv=5)
gsearch5.fit(train.values,outcome)
gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_

//anaconda/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda

([mean: -0.76045, std: 0.00730, params: {'reg_alpha': 1e-10, 'reg_lambda': 0.5},
  mean: -0.75940, std: 0.00696, params: {'reg_alpha': 1e-10, 'reg_lambda': 1},
  mean: -0.77397, std: 0.00769, params: {'reg_alpha': 1e-10, 'reg_lambda': 25},
  mean: -0.78058, std: 0.00809, params: {'reg_alpha': 1e-10, 'reg_lambda': 50},
  mean: -0.76045, std: 0.00730, params: {'reg_alpha': 1e-08, 'reg_lambda': 0.5},
  mean: -0.75940, std: 0.00696, params: {'reg_alpha': 1e-08, 'reg_lambda': 1},
  mean: -0.77397, std: 0.00769, params: {'reg_alpha': 1e-08, 'reg_lambda': 25},
  mean: -0.78058, std: 0.00809, params: {'reg_alpha': 1e-08, 'reg_lambda': 50},
  mean: -0.76070, std: 0.00732, params: {'reg_alpha': 1e-05, 'reg_lambda': 0.5},
  mean: -0.75940, std: 0.00696, params: {'reg_alpha': 1e-05, 'reg_lambda': 1},
  mean: -0.77405, std: 0.00781, params: {'reg_alpha': 1e-05, 'reg_lambda': 25},
  mean: -0.78058, std: 0.00809, params: {'reg_alpha': 1e-05, 'reg_lambda': 50},
  mean: -0.76090, std: 0.00712, params: 

'reg_alpha': 1e-05, 'reg_lambda': 1},
 -0.75940337024263005

In [149]:
xgb_tuned = XGBClassifier(learning_rate =0.01, n_estimators=5000, max_depth=8,
 min_child_weight=1, gamma=0.9, subsample=0.8, colsample_bytree=0.75,reg_alpha =1e-05, reg_lambda = 1, 
 objective= 'multi:softprob',nthread=4, scale_pos_weight=1,seed=27)
xgb_tuned = xgb_tuned.fit(train.values,outcome,eval_metric = 'mlogloss')

In [152]:
train_y_prediction = xgb_tuned.predict(train.values)
train_y_perdprob = xgb_tuned.predict_proba(train.values)

In [153]:
metrics.accuracy_score(outcome,train_y_prediction)

0.85869280556698713

array([ 0.53642768,  0.00335125,  0.23850651,  0.05440159,  0.16731298], dtype=float32)

([mean: 0.84479, std: 0.02617, params: {'min_child_weight': 1, 'max_depth': 3},
  mean: 0.84502, std: 0.02447, params: {'min_child_weight': 3, 'max_depth': 3},
  mean: 0.84678, std: 0.02293, params: {'min_child_weight': 5, 'max_depth': 3},
  mean: 0.84284, std: 0.02511, params: {'min_child_weight': 1, 'max_depth': 5},
  mean: 0.84399, std: 0.02894, params: {'min_child_weight': 3, 'max_depth': 5},
  mean: 0.84755, std: 0.02700, params: {'min_child_weight': 5, 'max_depth': 5},
  mean: 0.83168, std: 0.02229, params: {'min_child_weight': 1, 'max_depth': 7},
  mean: 0.83924, std: 0.02286, params: {'min_child_weight': 3, 'max_depth': 7},
  mean: 0.84213, std: 0.02765, params: {'min_child_weight': 5, 'max_depth': 7},
  mean: 0.82738, std: 0.02106, params: {'min_child_weight': 1, 'max_depth': 9},
  mean: 0.83525, std: 0.01870, params: {'min_child_weight': 3, 'max_depth': 9},
  mean: 0.84548, std: 0.02843, params: {'min_child_weight': 5, 'max_depth': 9}],
 {'max_depth': 5, 'min_child_weight': 5

([mean: 0.83887, std: 0.00801, params: {'min_child_weight': 1, 'max_depth': 3},
  mean: 0.83946, std: 0.00887, params: {'min_child_weight': 3, 'max_depth': 3},
  mean: 0.83904, std: 0.00746, params: {'min_child_weight': 5, 'max_depth': 3},
  mean: 0.84921, std: 0.00670, params: {'min_child_weight': 1, 'max_depth': 5},
  mean: 0.84857, std: 0.00750, params: {'min_child_weight': 3, 'max_depth': 5},
  mean: 0.84948, std: 0.00621, params: {'min_child_weight': 5, 'max_depth': 5},
  mean: 0.84923, std: 0.00802, params: {'min_child_weight': 1, 'max_depth': 7},
  mean: 0.85145, std: 0.00546, params: {'min_child_weight': 3, 'max_depth': 7},
  mean: 0.85128, std: 0.00743, params: {'min_child_weight': 5, 'max_depth': 7},
  mean: 0.84574, std: 0.00627, params: {'min_child_weight': 1, 'max_depth': 9},
  mean: 0.84935, std: 0.00600, params: {'min_child_weight': 3, 'max_depth': 9},
  mean: 0.84844, std: 0.00626, params: {'min_child_weight': 5, 'max_depth': 9}],
 {'max_depth': 7, 'min_child_weight': 3

In [156]:
test = pd.read_csv('cleaned_test.csv')
test_prediction = xgb_tuned.predict_proba(test.values)

In [157]:
test_prediction = pd.DataFrame(test_prediction_tuned)
test_prediction.columns = ['Adoption','Died','Euthanasia','Return_to_owner', 'Transfer']
ID = pd.read_csv('test.csv')
test_prediction['ID'] = ID['ID']
test_prediction = test_prediction[['ID', 'Adoption','Died','Euthanasia','Return_to_owner', 'Transfer']]
test_prediction.to_csv('xgb_prediction_tuned.csv')

,index,ID,Adoption,Died,Euthanasia,Return_to_owner,Transfer
0,0,1,0.008382,0.000023,0.087456,0.204877,0.699262
1,1,2,0.664976,0.000028,0.006026,0.247550,0.081419
2,2,3,0.508407,0.000091,0.002998,0.249212,0.239292
3,3,4,0.299198,0.000117,0.025895,0.272709,0.402081
4,4,5,0.451377,0.000597,0.001756,0.495272,0.050998
5,5,6,0.325008,0.000169,0.028970,0.466594,0.179260
6,6,7,0.388458,0.001444,0.268615,0.301554,0.039929
7,7,8,0.881975,0.002712,0.012389,0.024750,0.078173
8,8,9,0.132148,0.000003,0.003028,0.803359,0.061462
9,9,10,0.657656,0.000144,0.041290,0.256422,0.044488


In [45]:
total.head()

,AnimalType,AgeCode,ColorCode,BreedCode,IntactOrNot,NameCode,Hour,Weekday,Day of Month,Month,Year,OutcomeType
0,1,1,0,0,1.0,1,18,2,12,2,2014,Return_to_owner
1,0,1,0,0,1.0,1,12,6,13,10,2013,Euthanasia
2,1,1,0,0,1.0,1,12,5,31,1,2015,Adoption
3,0,0,0,0,0.0,0,19,4,11,7,2014,Transfer
4,1,1,1,0,1.0,0,12,4,15,11,2013,Transfer


In [28]:
train.head()

,AnimalType,AgeCode,ColorCode,BreedCode,IntactOrNot,NameCode,Hour,Weekday,Day of Month,Month,Year
0,1,1,0,0,1.0,1,18,2,12,2,2014
1,0,1,0,0,1.0,1,12,6,13,10,2013
2,1,1,0,0,1.0,1,12,5,31,1,2015
3,0,0,0,0,0.0,0,19,4,11,7,2014
4,1,1,1,0,1.0,0,12,4,15,11,2013


In [89]:
outcome.shape

(26729, 5)